In [1]:
!pip install nnsight plotly

In [2]:
!pip install git+https://github.com/liuhaozhe6788/crosscoder_vis.git

  Cloning https://github.com/liuhaozhe6788/crosscoder_vis.git to /tmp/pip-req-build-lxe5w4s4
  Running command git clone --filter=blob:none --quiet https://github.com/liuhaozhe6788/crosscoder_vis.git /tmp/pip-req-build-lxe5w4s4
  Resolved https://github.com/liuhaozhe6788/crosscoder_vis.git to commit aff34b5129caa74161cddd86b63a037e4dbe4e89
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
from typing import Optional, Union
from huggingface_hub import hf_hub_download, login
import json
import einops
import os
from typing import NamedTuple
from nnsight import LanguageModel
import numpy as np
import pandas as pd
import plotly.express as px

In [4]:
# HF_TOKEN = os.getenv("HF_TOKEN")
# login(HF_TOKEN)

In [5]:
torch.set_grad_enabled(False)

torch.autograd.grad_mode.set_grad_enabled(mode=False)

In [6]:
base_model = LanguageModel('mistralai/Mistral-7B-Instruct-v0.3', device_map='cuda:0', dtype=torch.bfloat16)
chat_model = LanguageModel('liuhaozhe6788/mistralai_Mistral-7B-Instruct-v0.3-FinQA-lora', device_map='cuda:0', dtype=torch.bfloat16)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
!pip install gdown
!gdown --fuzzy https://drive.google.com/file/d/1wEwq7YvskXf-lhbkdD1RqKpEJDu5bBpC/view?usp=sharinghttps://drive.google.com/file/d

Downloading...
From: https://drive.google.com/uc?id=1wEwq7YvskXf-lhbkdD1RqKpEJDu5bBpC
To: /content/finqa_test_generated_filtered.csv
100% 4.83M/4.83M [00:00<00:00, 157MB/s]


In [8]:
def prepare_text_data():
    data = pd.read_csv("finqa_test_generated_filtered.csv")
    full_text_data = data.apply(lambda x: x["prompt"] + x["generated_code"], axis=1)

    all_text_data = full_text_data.tolist()
    return all_text_data

all_texts = prepare_text_data()

# Loading the crosscoder

In [9]:
DTYPES = {"fp32": torch.float32, "fp16": torch.float16, "bf16": torch.bfloat16}

class LossOutput(NamedTuple):
    # loss: torch.Tensor
    l2_loss: torch.Tensor
    l1_loss: torch.Tensor
    l0_loss: torch.Tensor
    explained_variance: torch.Tensor
    explained_variance_A: torch.Tensor
    explained_variance_B: torch.Tensor

class CrossCoder_demo(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        d_hidden = self.cfg["dict_size"]
        d_in = self.cfg["d_in"]
        self.dtype = DTYPES[self.cfg["enc_dtype"]]
        torch.manual_seed(self.cfg["seed"])
        # hardcoding n_models to 2
        self.W_enc = nn.Parameter(
            torch.empty(2, d_in, d_hidden, dtype=self.dtype)
        )
        self.W_dec = nn.Parameter(
            torch.nn.init.normal_(
                torch.empty(
                    d_hidden, 2, d_in, dtype=self.dtype
                )
            )
        )
        self.W_dec = nn.Parameter(
            torch.nn.init.normal_(
                torch.empty(
                    d_hidden, 2, d_in, dtype=self.dtype
                )
            )
        )
        # Make norm of W_dec 0.1 for each column, separate per layer
        self.W_dec.data = (
            self.W_dec.data / self.W_dec.data.norm(dim=-1, keepdim=True) * self.cfg["dec_init_norm"]
        )
        # Initialise W_enc to be the transpose of W_dec
        self.W_enc.data = einops.rearrange(
            self.W_dec.data.clone(),
            "d_hidden n_models d_model -> n_models d_model d_hidden",
        )
        self.b_enc = nn.Parameter(torch.zeros(d_hidden, dtype=self.dtype))
        self.b_dec = nn.Parameter(
            torch.zeros((2, d_in), dtype=self.dtype)
        )
        self.d_hidden = d_hidden

        self.to(self.cfg["device"])
        self.save_dir = None
        self.save_version = 0

    def encode(self, x, apply_relu=True):
        # x: [batch, n_models, d_model]
        x_enc = einops.einsum(
            x,
            self.W_enc,
            "batch n_models d_model, n_models d_model d_hidden -> batch d_hidden",
        )
        if apply_relu:
            acts = F.relu(x_enc + self.b_enc)
        else:
            acts = x_enc + self.b_enc
        acts = self.mask_acts_batchtopk(acts)
        return acts

    def decode(self, acts):
        # acts: [batch, d_hidden]
        acts_dec = einops.einsum(
            acts,
            self.W_dec,
            "batch d_hidden, d_hidden n_models d_model -> batch n_models d_model",
        )
        return acts_dec + self.b_dec

    def forward(self, x):
        # x: [batch, n_models, d_model]
        acts = self.encode(x)
        return self.decode(acts)

    def mask_acts_batchtopk(self, acts):
        # acts: [batch, d_hidden]
        if self.cfg["batch_topk"] is not None:
            # Get topk across the whole batch
            acts_flat = acts.flatten()
            _, topk_indices = torch.topk(acts_flat, k=self.cfg["batch_topk"] * acts.shape[0], dim=-1)
            # Create a boolean mask from the indices
            mask_flat = torch.zeros_like(acts_flat, dtype=torch.bool)
            mask_flat[topk_indices] = True
            mask = mask_flat.reshape_as(acts)
            acts = torch.where(mask, acts, 0)
        return acts

    def get_losses(self, x):
        # x: [batch, n_models, d_model]
        x = x.to(self.dtype)
        acts = self.encode(x)
        x_reconstruct = self.decode(acts)
        diff = x_reconstruct.float() - x.float()
        squared_diff = diff.pow(2)
        l2_per_batch = einops.reduce(squared_diff, 'batch n_models d_model -> batch', 'sum')
        l2_loss = l2_per_batch.mean()

        total_variance = einops.reduce((x - x.mean(0)).pow(2), 'batch n_models d_model -> batch', 'sum')
        explained_variance = 1 - l2_per_batch / total_variance

        per_token_l2_loss_A = (x_reconstruct[:, 0, :] - x[:, 0, :]).pow(2).sum(dim=-1).squeeze()
        total_variance_A = (x[:, 0, :] - x[:, 0, :].mean(0)).pow(2).sum(-1).squeeze()
        explained_variance_A = 1 - per_token_l2_loss_A / total_variance_A

        per_token_l2_loss_B = (x_reconstruct[:, 1, :] - x[:, 1, :]).pow(2).sum(dim=-1).squeeze()
        total_variance_B = (x[:, 1, :] - x[:, 1, :].mean(0)).pow(2).sum(-1).squeeze()
        explained_variance_B = 1 - per_token_l2_loss_B / total_variance_B

        decoder_norms = self.W_dec.norm(dim=-1)
        # decoder_norms: [d_hidden, n_models]
        total_decoder_norm = einops.reduce(decoder_norms, 'd_hidden n_models -> d_hidden', 'sum')
        l1_loss = (acts * total_decoder_norm[None, :]).sum(-1).mean(0)

        l0_loss = (acts>0).float().sum(-1).mean()

        return LossOutput(l2_loss=l2_loss, l1_loss=l1_loss, l0_loss=l0_loss, explained_variance=explained_variance, explained_variance_A=explained_variance_A, explained_variance_B=explained_variance_B)

    @classmethod
    def load_from_hf(
        cls,
        repo_id: str = "liuhaozhe6788/crosscoder-model-diff-mistral-7b-instruct-v0.3_finQA_lora_topk_100",
        device: Optional[Union[str, torch.device]] = None
    ) -> "CrossCoder_demo":
        """
        Load CrossCoder_demo weights and config from HuggingFace.

        Args:
            repo_id: HuggingFace repository ID
            path: Path within the repo to the weights/config
            model: The transformer model instance needed for initialization
            device: Device to load the model to (defaults to cfg device if not specified)

        Returns:
            Initialized CrossCoder_demo instance
        """

        # Download config and weights
        config_path = hf_hub_download(
            repo_id=repo_id,
            filename=f"cfg.json"
        )
        weights_path = hf_hub_download(
            repo_id=repo_id,
            filename=f"model.pt"
        )

        # Load config
        with open(config_path, 'r') as f:
            cfg = json.load(f)

        # Override device if specified
        if device is not None:
            cfg["device"] = str(device)

        # Initialize CrossCoder_demo with config
        instance = cls(cfg)

        # Load weights
        state_dict = torch.load(weights_path, map_location=cfg["device"])
        instance.load_state_dict(state_dict["model_state_dict"])

        return instance

In [10]:
cross_coder = CrossCoder_demo.load_from_hf()

# Replicating Anthropic results

In [11]:
norms = cross_coder.W_dec.norm(dim=-1)
norms.shape

torch.Size([16384, 2])

In [12]:
relative_norms = norms[:, 1] / norms.sum(dim=-1)
relative_norms.shape

torch.Size([16384])

In [13]:
relative_norms_np = relative_norms.detach().cpu().numpy()

In [14]:
k = 100
idx = np.argpartition(relative_norms_np, -k)[-k:]  # Indices not sorted
idx[np.argsort(relative_norms_np[idx])][::-1]  # Indices sorted by value from largest to smallest

array([ 3086, 12790, 11031,  9553,  9606,  8180,  7932,  9947, 13217,
        3711,  2644, 11516,  2937,  7899,  9354,   744,  7441, 11526,
       12739, 14031,  4612,  1001,   689,  6809, 11622,  3883,  1073,
        6357,  4752,  6905,  6698,  5017,  2801, 14329,  5551,   576,
        3488, 14024,  6678,  7994,  3770, 15253,   803, 11825, 12920,
       16168, 12964,  9346,   424,  7518,  5016, 10063,  4109,   614,
       15831,  4983,  3444,  8882,  4754, 14683, 12583,  2702, 11400,
         290, 12240, 14643,  5664,  2919,  8976, 14513,  5589, 13644,
       12504,  5823,  8816,  7450, 12088, 11151,  6154,  9031,  1201,
       14405, 13484,  8217, 10714, 15441,  1789, 11064,  6943,   506,
       14473,  6004,  6839,  3971,  4773,  5843, 10849, 15638, 12582,
        4633])

In [15]:
idx = np.argpartition(relative_norms_np, k)[:k]  # Indices not sorted

idx[np.argsort(relative_norms_np[idx])]  # Indices sorted by value from smallest to largest

array([ 1436,  4902, 15811,  2733,   737,  8031, 13346,  6929, 14534,
         605,  8320,  6488, 16318, 14058,  8604, 10139,  3725,  6466,
       13341, 11141,  1596, 11152, 13738, 10231,  3618,   779, 16197,
        3797,    65, 10830,  6645, 14456, 10821, 11984,  4661,  5111,
        8432,  1117, 12881, 13990,  5145,  4172,  9732,  7537,  9272,
        5758, 13295, 15096, 12862,  2866,  1387,  6486, 10792, 12589,
       14634, 16093, 12057,  4718,  2498,  1870, 12612,  3865, 12565,
        2139,  2430,  6956,    68, 10350, 15617, 14057,  8983, 12467,
        8998, 12053,  1832,  2774, 13941, 10199, 15313, 10058,  6403,
       14022, 15807,  4950,  5466, 10325, 12249,  6019, 16104,  2220,
         383,  5847,  7715,  3152, 14337,  8027, 11278, 11496,  2512,
        8789])

In [16]:
fig = px.histogram(
    relative_norms.detach().cpu().numpy(),
    title="Mistral 7b Instruct v0.3 vs FinQA FT Model Diff",
    labels={"value": "Relative decoder norm strength"},
    nbins=200,
)

fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="Number of Latents")

# Update x-axis ticks
fig.update_xaxes(
    tickvals=[0, 0.25, 0.5, 0.75, 1.0],
    ticktext=['0', '0.25', '0.5', '0.75', '1.0']
)

fig.show()

In [17]:
shared_latent_mask = (relative_norms < 0.7) & (relative_norms > 0.3)
shared_latent_mask.shape

torch.Size([16384])

In [18]:
cosine_sims = (cross_coder.W_dec[:, 0, :] * cross_coder.W_dec[:, 1, :]).sum(dim=-1) / (cross_coder.W_dec[:, 0, :].norm(dim=-1) * cross_coder.W_dec[:, 1, :].norm(dim=-1))
cosine_sims.shape

torch.Size([16384])

In [19]:
fig = px.histogram(
    cosine_sims[shared_latent_mask].to(torch.float32).detach().cpu().numpy(),
    #title="Cosine similarity of decoder vectors between models",
    log_y=True,  # Sets the y-axis to log scale
    range_x=[-1, 1],  # Sets the x-axis range from -1 to 1
    nbins=100,  # Adjust this value to change the number of bins
    labels={"value": "Cosine similarity of decoder vectors between models"}
)

fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="Number of Latents (log scale)")

fig.show()

In [20]:
import copy
folded_cross_coder = copy.deepcopy(cross_coder)

base_estimated_scaling_factor = 27.489933013916016
chat_estimated_scaling_factor = 27.12582778930664

def fold_activation_scaling_factor(cross_coder, base_scaling_factor, chat_scaling_factor):
    cross_coder.W_enc.data[0, :, :] = cross_coder.W_enc.data[0, :, :] * base_scaling_factor
    cross_coder.W_enc.data[1, :, :] = cross_coder.W_enc.data[1, :, :] * chat_scaling_factor

    # cross_coder.W_dec.data[:, 0, :] = cross_coder.W_dec.data[:, 0, :] / base_scaling_factor
    # cross_coder.W_dec.data[:, 1, :] = cross_coder.W_dec.data[:, 1, :] / chat_scaling_factor

    # cross_coder.b_dec.data[0, :] = cross_coder.b_dec.data[0, :] / base_scaling_factor
    # cross_coder.b_dec.data[1, :] = cross_coder.b_dec.data[1, :] / chat_scaling_factor
    return cross_coder

folded_cross_coder = fold_activation_scaling_factor(folded_cross_coder, base_estimated_scaling_factor, chat_estimated_scaling_factor)


# Generating latent dashboards

In [21]:
from sae_vis.model_fns import CrossCoderConfig, CrossCoder_vis
from sae_vis.data_config_classes import SaeVisConfig

In [22]:
encoder_cfg = CrossCoderConfig(d_in=base_model.config.hidden_size, d_hidden=cross_coder.cfg["dict_size"], apply_b_dec_to_input=False)
sae_vis_cross_coder = CrossCoder_vis(encoder_cfg)
sae_vis_cross_coder.load_state_dict(folded_cross_coder.state_dict())
sae_vis_cross_coder = sae_vis_cross_coder.to("cuda:0")
sae_vis_cross_coder = sae_vis_cross_coder.to(torch.bfloat16)

In [23]:
test_feature_idx = [3086]
sae_vis_config = SaeVisConfig(
    hook_layer = 16,
    features = test_feature_idx,
    verbose = True,
    minibatch_size_texts=1,
    minibatch_size_features=16,
)

In [24]:
from sae_vis.data_storing_fns import SaeVisData
crosscoder_vis_data = SaeVisData.create(
    encoder = sae_vis_cross_coder,
    encoder_B = None,
    model_A = base_model,
    model_B = chat_model,
    texts = all_texts[:128], # in practice, better to use more data
    cfg = sae_vis_config,
)

Forward passes to cache data for vis:   0%|          | 0/128 [00:00<?, ?it/s]

Extracting vis data from cached data:   0%|          | 0/1 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/eindex/indexing.py:288: UserWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)

/usr/local/lib/python3.12/dist-packages/eindex/indexing.py:292: UserWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:345.)

/usr/local/lib/python3.12/dist-packages/eindex/indexing.py:296: UserWarning:

Using a non-tuple sequ

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃ Task                                           ┃ Time   ┃ Pct % ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ (1) Initialization                             │ 0.44s  │ 0.6%  │
│ (2) Forward passes to gather model activations │ 45.59s │ 65.9% │
│ (3) Computing feature acts from model acts     │ 22.90s │ 33.1% │
│ (4) Getting data for tables                    │ 0.00s  │ 0.0%  │
│ (5) Getting data for histograms                │ 0.06s  │ 0.1%  │
│ (6) Getting data for sequences                 │ 0.09s  │ 0.1%  │
│ (7) Getting data for quantiles                 │ 0.12s  │ 0.2%  │
└────────────────────────────────────────────────┴────────┴───────┘

In [25]:
import os
import http
import socketserver
import threading
from google.colab import output

PORT = 8000

def display_vis_inline(filename: str, height: int = 850):
    '''
    Displays the HTML files in Colab. Uses global `PORT` variable defined in prev cell, so that each
    vis has a unique port without having to define a port within the function.
    '''
    global PORT

    def serve(directory):
        os.chdir(directory)

        # Create a handler for serving files
        handler = http.server.SimpleHTTPRequestHandler

        # Create a socket server with the handler
        with socketserver.TCPServer(("", PORT), handler) as httpd:
            print(f"Serving files from {directory} on port {PORT}")
            httpd.serve_forever()

    thread = threading.Thread(target=serve, args=("/content",))
    thread.start()

    output.serve_kernel_port_as_iframe(PORT, path=f"/{filename}", height=height, cache_in_notebook=True)

    PORT += 1

filename = "_feature_vis_demo.html"
crosscoder_vis_data.save_feature_centric_vis(filename)

display_vis_inline(filename)

Saving feature-centric vis:   0%|          | 0/1 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

Serving files from /content on port 8001
